In [9]:
import cv2
import pyautogui
import pydirectinput
from time import time
from math import hypot
import mediapipe as mp
import matplotlib.pyplot as plt

pydirectinput.PAUSE = 0

In [10]:
mp_pose = mp.solutions.pose
 
pose_image = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5, model_complexity=1)
 
pose_video = mp_pose.Pose(static_image_mode=False, model_complexity=1, min_detection_confidence=0.7,
                          min_tracking_confidence=0.7)

mp_drawing = mp.solutions.drawing_utils 

In [11]:
def put_fps(image, time1):
    time2 = time()
    if (time2 - time1) > 0:
        frames_per_second = 1.0 / (time2 - time1)
        cv2.putText(image, 'FPS: {}'.format(int(frames_per_second)), (10, 30),cv2.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 3)
    return time2
    

In [12]:
def detect_pose(image, draw=False):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose_video.process(image)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    if draw:
        mp_drawing.draw_landmarks(image=image, landmark_list=results.pose_landmarks,
                                  connections=mp_pose.POSE_CONNECTIONS,
                                  landmark_drawing_spec=mp_drawing.DrawingSpec(color=(255,255,255),
                                                                               thickness=3, circle_radius=3),
                                  connection_drawing_spec=mp_drawing.DrawingSpec(color=(49,125,237),
                                                                               thickness=2, circle_radius=2))
    return image, results

In [13]:
def control_game(image, landmarks):
    height, width, _ = image.shape

    left_wrist = int(landmarks[mp_pose.PoseLandmark.LEFT_WRIST].y * height)
    right_wrist = int(landmarks[mp_pose.PoseLandmark.RIGHT_WRIST].y * height)
    nose_pose = int(landmarks[mp_pose.PoseLandmark.NOSE].y * height)

    JUMP_THRESH = 130
    IDLE_THRESH = 290

    control = 'idle'
    if nose_pose < JUMP_THRESH and (left_wrist < IDLE_THRESH or right_wrist < IDLE_THRESH):
        control = 'x'
    elif right_wrist < IDLE_THRESH and left_wrist > right_wrist:
        control = 'a'
    elif left_wrist < IDLE_THRESH and left_wrist < right_wrist:
        control = 's'
    elif nose_pose < JUMP_THRESH:
        control = 'up'
    elif nose_pose > IDLE_THRESH:
        control = 'down'

    cv2.line(image, (0, JUMP_THRESH), (width - 1, JUMP_THRESH), (255, 255, 255), 2)
    cv2.line(image, (0, IDLE_THRESH), (width - 1, IDLE_THRESH), (255, 255, 255), 2)
    cv2.putText(image, 'ACTION: {}'.format(control), (400, 30),cv2.FONT_HERSHEY_PLAIN, 2, (255, 0, 0), 3)
    return image, control

In [14]:
def press(key):
    pyautogui.keyDown(key)
    pyautogui.sleep(0.01)
    pyautogui.keyUp(key)

In [15]:
def press_key(control, pause):
    PAUSE_DURATION = 2
    if control == 's' and pause <= 0:
        press('s')
        pause = PAUSE_DURATION
    elif control == 'a' and pause <= 0:
        press('a')
        pause = PAUSE_DURATION
    elif control == 'x' and pause <= 0:
        press('x')
        pause = PAUSE_DURATION
    elif control == 'up' and pause <= 0:
        press('up')
        pause = PAUSE_DURATION
    elif control == 'down' and pause <= 0:
        press('down')
        pause = PAUSE_DURATION
    return pause

In [16]:
cap = cv2.VideoCapture(0)

cv2.namedWindow('Play Active', cv2.WINDOW_NORMAL)

time1 = time()
pause = 0
while cap.isOpened():
    
    ok, frame = cap.read()
    
    if not ok:
        continue
    
    frame = cv2.flip(frame, 1)
    # control = 's'
    
    # frame_height, frame_width, _ = frame.shape
    frame, results = detect_pose(frame, True)

    try:
        landmarks = results.pose_landmarks.landmark
        frame, control = control_game(frame, landmarks)
        pause = press_key(control, pause)
    except:
        pass
    
    time1 = put_fps(frame, time1)

    cv2.imshow('Play Active', frame)

    pause -= 1
    k = cv2.waitKey(15) & 0xFF
    if(k == 27):
        break
       
cap.release()
cv2.destroyAllWindows() 